# Exercise: Learning to recognise touch sounds

## A data processing pipeline

## Step 1: Loading the data
## Step 2: Pre-processing
## Step 3: Feature extraction
## Step 4: Building a classifier
## Step 6: Evaluating performance
## Step 7: A better classifier
## Step 8: Better evaluation methods
## Step 9: Experiment